In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from simple_salesforce import Salesforce, format_soql, format_external_id
import paramiko
import datetime
from datetime import date
import re
from collections import OrderedDict
import simpler_sf
simpler_sf.simple_salesforce()
import simple_salesforce

c:\Users\jmartinez\.conda\envs\salesforce\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
with open("login.json", "r") as j:
    misDatos = json.load(j)

In [3]:
#para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
#para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

In [4]:
fecha = datetime.date.today()
dia_hoy = str(fecha.day)
mes_hoy = str(fecha.month)
ano_hoy = str(fecha.year)
if len(mes_hoy) == 1:
    mes_hoy = "0" + mes_hoy
if len(dia_hoy) == 1:
    dia_hoy = "0" + dia_hoy
archivoDia = dia_hoy + mes_hoy + ano_hoy

nombreCarpetaHost = "/care_assistance/BCI_a_CAREASSIST/PRODUCCION" + "/"
nombreArchivoHost = "VentaCARE_ASSISTANCE_" + str(archivoDia) + ".xlsx"
#nombreArchivoHost = "VentaCARE_ASSISTANCE_" + '22112023' + ".xlsx"
descargaDeArchivo = nombreCarpetaHost + nombreArchivoHost

# Direccion descarga local
direccionDescargaLocal = (
    "F:/Procesos Automaticos/BCI Canal Individual/" + nombreArchivoHost
)

print(fecha)
print(archivoDia)
print(nombreArchivoHost)
print(descargaDeArchivo)

2024-01-03
03012024
VentaCARE_ASSISTANCE_03012024.xlsx
/care_assistance/BCI_a_CAREASSIST/PRODUCCION/VentaCARE_ASSISTANCE_03012024.xlsx


In [5]:
host = "sftp.bciseguros.cl"
puerto = 22
transporte = paramiko.Transport((host, puerto))
usuario = "pcerda_cas"
clave = "wB6tfCRc"
transporte.connect(None, usuario, clave)
sftp = paramiko.SFTPClient.from_transport(transporte)

Coneccion a carpeta sft y rescate de archivo


In [6]:
# Download
filepath = descargaDeArchivo
localpath = direccionDescargaLocal
sftp.get(filepath, localpath)

In [7]:
if sftp:
    sftp.close()
if transporte:
    transporte.close()

Inicio Manipulacion de Excel descargado


In [8]:
archivoBCI = pd.read_excel(direccionDescargaLocal)
archivoBCI.shape


c:\Users\jmartinez\.conda\envs\salesforce\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


(10988, 17)

In [9]:
archivoBCI.dtypes


POLIZA                       int64
FECINIVIG           datetime64[ns]
FECFINVIG           datetime64[ns]
CODPROD                     object
CODPLAN                     object
DESCRIPCION_PLAN            object
RUT_TITULAR                  int64
DV_TITULAR                  object
RUT_ASEGURADO              float64
DV_ASEGURADO                object
SEXO                        object
PARENTESCO                  object
NOMBRE_ASEGURADO            object
FECHA_NACIMIENTO    datetime64[ns]
CORREO                      object
DIREC                       object
CELULAR                    float64
dtype: object

In [10]:
archivoBCI.RUT_ASEGURADO.fillna(archivoBCI["RUT_TITULAR"], inplace=True)
archivoBCI.DV_ASEGURADO.fillna(archivoBCI["DV_TITULAR"], inplace=True)
archivoBCI.SEXO.fillna("0", inplace=True)
archivoBCI.CORREO.fillna("", inplace=True)
archivoBCI.CELULAR.fillna("0", inplace=True)
archivoBCI.DIREC.fillna("", inplace=True)
archivoBCI.NOMBRE_ASEGURADO.fillna("NN NN", inplace=True)
archivoBCI["RUT_ASEGURADO"] = archivoBCI["RUT_ASEGURADO"].apply(np.int64)
archivoBCI["CELULAR"] = archivoBCI["CELULAR"].astype(int)
archivoBCI["CELULAR"] = archivoBCI["CELULAR"].apply(np.int64)


archivoBCI.head(-50)

,POLIZA,FECINIVIG,FECFINVIG,CODPROD,CODPLAN,DESCRIPCION_PLAN,RUT_TITULAR,DV_TITULAR,RUT_ASEGURADO,DV_ASEGURADO,SEXO,PARENTESCO,NOMBRE_ASEGURADO,FECHA_NACIMIENTO,CORREO,DIREC,CELULAR
0,2611887,2023-06-19,2024-06-18,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,19201148,5,19201148,5,FEMENINO,TITULAR FEMENINO,MARIA DEL CARMEN PAILLACAR LOPEZ,1988-09-13,MARIAPAILLACARLOPEZ@GMAIL.COM,S/D,952183462
1,2611900,2023-06-20,2024-06-19,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,8769586,7,8769586,7,FEMENINO,TITULAR MASCULINO,OLGA VIVIANA DEL CARMEN GOMEZ AEDO,1959-11-02,VIVIANA.GOMEZ2@GMAIL.COM,"CALLE SAN PEDRO 620, SAN PEDRO DE LA PAZ",979447792
2,2611969,2023-06-20,2024-06-19,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,16902476,6,16902476,6,FEMENINO,TITULAR FEMENINO,SUSAN RANHDY BRAVO ANDERSON,1988-07-08,SUSAN.BRAVOA@GMAIL.COM,FRANCISCO DE LAZCANO 13576,974865798
3,2611912,2023-06-20,2024-06-19,SAIN,M1,SALUD INDIVIDUAL PLAN 60 WEB,13000967,0,13000967,0,FEMENINO,TITULAR FEMENINO,EDITH MARGOT DEL RÍO OYARZO,1975-11-15,edithdelrio@gmail.com,PASAJE CORONA NRO 28 DEPTO VILLA FUERTE REAL,977606025
4,2611922,2023-06-20,2024-06-19,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,18112141,6,18112141,6,FEMENINO,TITULAR MASCULINO,BARBARA ALEJANDRA RICCI VALDES,1991-06-08,fayrus17@gmail.com,SANTIAGO,983740695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10933,2657398,2023-10-13,2024-10-12,SAIN,M1,SALUD INDIVIDUAL PLAN 60 WEB,18247591,2,18247591,2,FEMENINO,TITULAR MASCULINO,PAMELA PAZ SANDOVAL MARTINEZ,1992-11-04,PPSANDOVALMARTINEZ@GMAIL.COM,RIO LOA 143 LAMPA,993836541
10934,2657400,2023-10-13,2024-10-12,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,10591963,8,10591963,8,MASCULINO,TITULAR MASCULINO,MARCO AURELIO VIDAL ROJAS,1965-05-28,MARCO.VIDAL.R@GMAIL.COM,MECHUQUE 294,999786131
10935,2657977,2023-10-16,2024-10-15,SAIN,M1,SALUD INDIVIDUAL PLAN 60 WEB,16685029,0,16685029,0,FEMENINO,TITULAR MASCULINO,KAREN ANDREA NUNEZ MERINO,1987-05-14,KAREN.NUNEZMERINO@GMAIL.COM,AV REINA MAR 2613 TORRE 2 DEPTO 1501 IQUIQUE,966720800
10936,2657978,2023-10-16,2024-10-15,SAIN,M1,SALUD INDIVIDUAL PLAN 60 WEB,15356566,K,15356566,K,MASCULINO,TITULAR MASCULINO,PABLO ANDRES BEROY INOSTROZA,1983-03-18,beroy.pablo@yahoo.com,AVENIDA EJERCITO LIBERTADOR 4196 CASA 7,987617766


In [11]:
def isValid(email):
    if re.fullmatch(regex, email):
        return email
    else:
        return ""


def nombreFuncion(nombreCompleto):
    if len(nombreCompleto) >= 2:
        nombresJuntos = nombreCompleto[0] + " " + nombreCompleto[1]
        return nombresJuntos
    elif len(nombreCompleto) == 1:
        nombresJuntos = nombreCompleto[0]
        return nombresJuntos
    else:
        nombresJuntos = "NN NN"
        return nombresJuntos
    print(len(nombresJuntos) + nombresJuntos)


def apellidoFuncion(nombreCompleto):
    if len(nombreCompleto) >= 2:
        nombresJuntos = nombreCompleto[-2] + " " + nombreCompleto[-1]
        return nombresJuntos
    elif len(nombreCompleto) == 1:
        nombresJuntos = nombreCompleto[-1]
        return nombresJuntos
    else:
        nombresJuntos = "NN NN"
        return nombresJuntos
    print(len(nombresJuntos) + nombresJuntos)


""" def apellidoFuncion(nombreCompleto):
  nombreCompleto = str(nombreCompleto)
  apellidos = re.split('\W+', nombreCompleto)
  if len(apellidos)==1:
    return apellidos[0]
  elif len(apellidos)==2:
    nombresJuntos = apellidos[0] + " " + apellidos[1]
    return nombresJuntos
    
  elif len(apellidos)==3:
    archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]
    archivoBCI["apellido"] = apellidos[2]
  else:
    archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]
    archivoBCI["apellido"] = apellidos[2] + " " + apellidos[3] """

# archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]
# archivoBCI["apellido"] = apellidos[2] + " " + apellidos[3]
# print (apellidos)

' def apellidoFuncion(nombreCompleto):\n  nombreCompleto = str(nombreCompleto)\n  apellidos = re.split(\'\\W+\', nombreCompleto)\n  if len(apellidos)==1:\n    return apellidos[0]\n  elif len(apellidos)==2:\n    nombresJuntos = apellidos[0] + " " + apellidos[1]\n    return nombresJuntos\n    \n  elif len(apellidos)==3:\n    archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]\n    archivoBCI["apellido"] = apellidos[2]\n  else:\n    archivoBCI["nombre"] = apellidos[0] + " " + apellidos[1]\n    archivoBCI["apellido"] = apellidos[2] + " " + apellidos[3] '

In [12]:
archivoBCI["Rut Titular"] = archivoBCI["RUT_TITULAR"].astype(str) + archivoBCI[
    "DV_TITULAR"
].astype(str)
archivoBCI["Rut Titular"] = archivoBCI["Rut Titular"].str.replace(" ", "").str.upper()

archivoBCI["Rut Carga"] = archivoBCI["RUT_ASEGURADO"].astype(str) + archivoBCI[
    "DV_ASEGURADO"
].astype(str)
archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].str.replace(" ", "").str.upper()

archivoBCI["poliza"] = archivoBCI["POLIZA"].astype(str)

general_Sexo = {
    "M": "Hombre",
    "m": "Hombre",
    "Masculino": "Hombre",
    "Femenino": "Mujer",
    "": "Prefiero no responder",
    "HOMBRE": "Hombre",
    "MUJER": "Mujer",
    "F": "Mujer",
    "O": "O",
    "Ambos": "Prefiero no responder",
    "N": "Prefiero no responder",
    "Otro": "Prefiero no responder",
    "n": "Prefiero no responder",
    "N": "Prefiero no responder",
    "FEMENINO": "Mujer",
    "f": "Mujer",
    "MASCULINO": "Hombre",
    "m": "Hombre",
    "M ": "Hombre",
    "F ": "Mujer",
    "Varón": "Hombre",
    "0": "",
}
archivoBCI.replace({"SEXO": general_Sexo}, inplace=True)
archivoBCI["sexo"] = archivoBCI["SEXO"].astype(str).str.title()

general_email = {
    "no informado": "",
}
archivoBCI["email"] = archivoBCI["CORREO"].astype(str).str.lower()
archivoBCI["email"] = archivoBCI["email"].apply(lambda y: isValid(y))
archivoBCI.replace({"email": general_email}, inplace=True)

general_direccion = {
    "no informado": "",
    "SN": "",
    "sn": "",
    "s/d": "",
    "1 1": "",
    "0 0": "",
    "sn 0": "",
}
archivoBCI["direccion"] = archivoBCI["DIREC"].astype(str).str.lower()
archivoBCI.replace({"direccion": general_direccion}, inplace=True)
archivoBCI["direccion"] = archivoBCI["direccion"].str.title()

general_Celular = {
    0: "",
}
archivoBCI.replace({"CELULAR": general_Celular}, inplace=True)
archivoBCI["celular"] = archivoBCI["CELULAR"].astype(str).str.lower()
archivoBCI["celular"] = archivoBCI["celular"].str.replace("NaN", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("Nan", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("-", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("No Informado", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("_", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("+", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace("nan", "")
archivoBCI["celular"] = archivoBCI["celular"].str.replace(" ", "")


archivoBCI["nombres"] = archivoBCI["NOMBRE_ASEGURADO"].str.split(n=2)
archivoBCI["apellidos"] = archivoBCI["NOMBRE_ASEGURADO"].str.rsplit(n=2)
archivoBCI["nombres"] = (
    archivoBCI["nombres"].apply(lambda y: nombreFuncion(y)).str.title()
)
archivoBCI["apellidos"] = (
    archivoBCI["apellidos"].apply(lambda y: apellidoFuncion(y)).str.title()
)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_8040\2221294489.py:70: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  archivoBCI["celular"] = archivoBCI["celular"].str.replace("+", "")


In [13]:
archivoBCI.head(-50)


,POLIZA,FECINIVIG,FECFINVIG,CODPROD,CODPLAN,DESCRIPCION_PLAN,RUT_TITULAR,DV_TITULAR,RUT_ASEGURADO,DV_ASEGURADO,...,CELULAR,Rut Titular,Rut Carga,poliza,sexo,email,direccion,celular,nombres,apellidos
0,2611887,2023-06-19,2024-06-18,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,19201148,5,19201148,5,...,952183462,192011485,192011485,2611887,Mujer,mariapaillacarlopez@gmail.com,,952183462,Maria Del,Paillacar Lopez
1,2611900,2023-06-20,2024-06-19,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,8769586,7,8769586,7,...,979447792,87695867,87695867,2611900,Mujer,viviana.gomez2@gmail.com,"Calle San Pedro 620, San Pedro De La Paz",979447792,Olga Viviana,Gomez Aedo
2,2611969,2023-06-20,2024-06-19,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,16902476,6,16902476,6,...,974865798,169024766,169024766,2611969,Mujer,susan.bravoa@gmail.com,Francisco De Lazcano 13576,974865798,Susan Ranhdy,Bravo Anderson
3,2611912,2023-06-20,2024-06-19,SAIN,M1,SALUD INDIVIDUAL PLAN 60 WEB,13000967,0,13000967,0,...,977606025,130009670,130009670,2611912,Mujer,edithdelrio@gmail.com,Pasaje Corona Nro 28 Depto Villa Fuerte Real,977606025,Edith Margot,Río Oyarzo
4,2611922,2023-06-20,2024-06-19,SAIN,M3,SALUD INDIVIDUAL PLAN 80 WEB,18112141,6,18112141,6,...,983740695,181121416,181121416,2611922,Mujer,fayrus17@gmail.com,Santiago,983740695,Barbara Alejandra,Ricci Valdes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10933,2657398,2023-10-13,2024-10-12,SAIN,M1,SALUD INDIVIDUAL PLAN 60 WEB,18247591,2,18247591,2,...,993836541,182475912,182475912,2657398,Mujer,ppsandovalmartinez@gmail.com,Rio Loa 143 Lampa,993836541,Pamela Paz,Sandoval Martinez
10934,2657400,2023-10-13,2024-10-12,SAIN,M2,SALUD INDIVIDUAL PLAN 70 WEB,10591963,8,10591963,8,...,999786131,105919638,105919638,2657400,Hombre,marco.vidal.r@gmail.com,Mechuque 294,999786131,Marco Aurelio,Vidal Rojas
10935,2657977,2023-10-16,2024-10-15,SAIN,M1,SALUD INDIVIDUAL PLAN 60 WEB,16685029,0,16685029,0,...,966720800,166850290,166850290,2657977,Mujer,karen.nunezmerino@gmail.com,Av Reina Mar 2613 Torre 2 Depto 1501 Iquique,966720800,Karen Andrea,Nunez Merino
10936,2657978,2023-10-16,2024-10-15,SAIN,M1,SALUD INDIVIDUAL PLAN 60 WEB,15356566,K,15356566,K,...,987617766,15356566K,15356566K,2657978,Hombre,beroy.pablo@yahoo.com,Avenida Ejercito Libertador 4196 Casa 7,987617766,Pablo Andres,Beroy Inostroza


In [14]:
# archivoBCI = archivoBCI[archivoBCI["RUT_ASEGURADO"]== "0"]
# archivoBCI.to_excel("F:/Procesos Automaticos/BCI Canal Individual/derty.xlsx",index=False)

In [15]:
def revisaDict(ver):
    numbers = ver
    for key, value in numbers.items():
        print(key, "->", value)
        if key == "IdentificationId__pc":
            return value


""" def revisaDict(ver):
    numbers = ver
    for key, value in numbers.items():
        print(key, "->", value)
        if key == "IdentificationId__pc":
            return value """


' def revisaDict(ver):\n    numbers = ver\n    for key, value in numbers.items():\n        print(key, "->", value)\n        if key == "IdentificationId__pc":\n            return value '

In [16]:
results = sf.query_all(
    """
    select Paciente__r.IdentificationId__pc from Paciente_Programas__c where padre__c = 'BCI Seguros Individuales'
   """
)
pacienteProgr = pd.DataFrame(results["records"])
# Pacientes_SF["nuevo"] = Paciente__r.OrderedDict.IdentificationId__pc
# String acctName = carol.Account.Name;

pacienteProgr["sfRut"] = pacienteProgr["Paciente__r"].apply(lambda y: revisaDict(y))

pacienteProgr["sfRut"] = pacienteProgr["sfRut"].astype(str).str.upper()

pacienteProgr = pacienteProgr[["sfRut",]]


pacienteProgr.shape


attributes -> OrderedDict([('type', 'Account'), ('url', '/services/data/v42.0/sobjects/Account/0018c00002mVtphAAC')])
IdentificationId__pc -> 196364803
attributes -> OrderedDict([('type', 'Account'), ('url', '/services/data/v42.0/sobjects/Account/0018c00002mVtpiAAC')])
IdentificationId__pc -> 154512675
attributes -> OrderedDict([('type', 'Account'), ('url', '/services/data/v42.0/sobjects/Account/0018c00002mVtpjAAC')])
IdentificationId__pc -> 243180856
attributes -> OrderedDict([('type', 'Account'), ('url', '/services/data/v42.0/sobjects/Account/0018c00002mVtpkAAC')])
IdentificationId__pc -> 14067401K
attributes -> OrderedDict([('type', 'Account'), ('url', '/services/data/v42.0/sobjects/Account/0018c00002mVtplAAC')])
IdentificationId__pc -> 268701028
attributes -> OrderedDict([('type', 'Account'), ('url', '/services/data/v42.0/sobjects/Account/0018c00002mVtsvAAC')])
IdentificationId__pc -> 101272702
attributes -> OrderedDict([('type', 'Account'), ('url', '/services/data/v42.0/sobjects/A

(3205, 1)

In [17]:
pacienteProgr["sfRut"] = pacienteProgr["sfRut"].astype(str).str.upper()
pacienteProgr["sfRut"] = pacienteProgr["sfRut"].str.replace(" ", "")
pacienteProgr["sfRut"] = pacienteProgr["sfRut"].str.replace("-", "")

archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].astype(str).str.upper()
archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].str.replace(" ", "")
archivoBCI["Rut Carga"] = archivoBCI["Rut Carga"].str.replace("-", "")

archivoBCI["RutEstaoNo"] = archivoBCI["Rut Carga"].isin(pacienteProgr["sfRut"])

archivoBCI = archivoBCI[archivoBCI["RutEstaoNo"] == False]

archivoBCI.drop(
    archivoBCI[
        (archivoBCI["nombres"] == "Nn Nn") & (archivoBCI["apellidos"] == "Nn Nn")
    ].index,
    inplace=True,
)
archivoBCI.drop_duplicates(["Rut Carga", "Rut Titular"], keep="last")

archivoBCI = archivoBCI[
    ["Rut Carga", "sexo", "direccion", "email", "celular", "nombres", "apellidos"]
]

archivoBCI = archivoBCI.rename(
    columns={
        "Rut Carga": "IdentificationId__pc",
        "sexo": "HealthCloudGA__Gender__pc",
        "direccion": "BillingStreet",
        "email": "PersonEmail",
        "celular": "Phone",
        "nombres": "FirstName",
        "apellidos": "LastName",
    }
)
archivoBCI["BillingCountry"] = "Chile"
archivoBCI["Identification_Type__c"] = "RUT"
archivoBCI["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
archivoBCI["RecordTypeId"] = "0128c000002Byp4AAC"
archivoBCI["Cuenta_validada__c"] = "True"
archivoBCI["Validaci_n__c"] = "Validada"
archivoBCI["Email_Seteado__c"] = np.where(
    archivoBCI["PersonEmail"] != "", "True", "False"
)
""" archivoBCI.to_excel(
    "F:/Procesos Automaticos/BCI Seguros Individuales/derty.xlsx", index=False
) """


archivoBCI.columns


Index(['IdentificationId__pc', 'HealthCloudGA__Gender__pc', 'BillingStreet',
       'PersonEmail', 'Phone', 'FirstName', 'LastName', 'BillingCountry',
       'Identification_Type__c', 'Pa_s_del_Paciente__c', 'RecordTypeId',
       'Cuenta_validada__c', 'Validaci_n__c', 'Email_Seteado__c'],
      dtype='object')

In [35]:
""" derty = sf.quick_search("153298939")
#print(derty)

revisaDict(derty)
 """
soql_query = """ select Id, Paciente__r.IdentificationId__pc from Paciente_Programas__c where padre__c = 'BCI Seguros Individuales' """
soql_result = sf.query_all(soql_query)
df = pd.DataFrame(soql_result.get("records"))
df = df.drop(columns=["attributes"])
df = df.drop(columns="Id")
list_of_records = df.to_dict("index")

""" derty = sf.quick_search("153298939")
print(derty) """

list_of_records[0]["Paciente__r"]["IdentificationId__pc"]

'196364803'

Weeeeeeeeeeeenooo


In [19]:
# rut = "124942012"
def traerId(rut):
    if (len(rut)>=8):
        busquedaDatos = "FIND {" + rut + "} RETURNING Account(Id,IdentificationId__pc)"
        derty = dict(sf.search(busquedaDatos))
        if derty["searchRecords"] != []:
            if derty["searchRecords"][0]["attributes"]["type"] == "Account":
                jote = derty["searchRecords"][0]["Id"]
                dertyd = derty["searchRecords"][0]["IdentificationId__pc"]
                print(jote + " " + dertyd)
                return jote
            else:
                print("no")
        else:
            print("vacio")
    else:
        return ""        

In [20]:
archivoBCI["Paciente__c"] = archivoBCI["IdentificationId__pc"].apply(lambda y: traerId(y))
archivoBCI["Programas_por_Empresas__c"] = "a1W8c000008TCWxEAO"
archivoBCI.head()

vacio
vacio
vacio
vacio
vacio
vacio
vacio
0018c00002lUypPAAS 155932708
0018c00002eVby1AAC 129053577
vacio
0018c00002oNzGZAA0 87121992
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
0018c00002efTHdAAM 145478464
0018c00002efTHeAAM 22519095K
0018c00002efTHfAAM 23641955K
vacio
vacio
vacio
vacio
0018c00002azMqfAAE 161165530
vacio
0018c00002ayTwjAAE 90969544
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
0018c00002bIY7xAAG 276971263
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
0018c00002ayntOAAQ 105278322
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio
vacio


,IdentificationId__pc,HealthCloudGA__Gender__pc,BillingStreet,PersonEmail,Phone,FirstName,LastName,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c,RecordTypeId,Cuenta_validada__c,Validaci_n__c,Email_Seteado__c,Paciente__c,Programas_por_Empresas__c
7934,190699242,Mujer,.,gab.cisternag@gmail.com,,Gabriela Alejandra,Cisterna Garrido,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True,None,a1W8c000008TCWxEAO
7935,199370790,Mujer,.,francisco.alcota.h@gmail.com,,Alexandra Alexis,Duran Castillo,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True,None,a1W8c000008TCWxEAO
7936,165940962,Hombre,.,francisco.alcota.h@gmail.com,972087289,Francisco Andres,Alcota Hidalgo,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True,None,a1W8c000008TCWxEAO
7937,182046884,Mujer,,,933141806,Yamilet Margarita,Altamirano Maldonado,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,False,None,a1W8c000008TCWxEAO
7938,160031239,Hombre,Diego Portales 860,xmaquinax@gmail.com,975084179,Torres Valderrama,Patricio Joaquin,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True,None,a1W8c000008TCWxEAO


In [21]:
archivoDeCarga = (
    "F:/Procesos Automaticos/BCI Canal Individual/" + str(archivoDia) + ".xlsx"
)

archivoBCI.to_excel(archivoDeCarga, index=False)

archivoBCI.head()

,IdentificationId__pc,HealthCloudGA__Gender__pc,BillingStreet,PersonEmail,Phone,FirstName,LastName,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c,RecordTypeId,Cuenta_validada__c,Validaci_n__c,Email_Seteado__c,Paciente__c,Programas_por_Empresas__c
7934,190699242,Mujer,.,gab.cisternag@gmail.com,,Gabriela Alejandra,Cisterna Garrido,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True,None,a1W8c000008TCWxEAO
7935,199370790,Mujer,.,francisco.alcota.h@gmail.com,,Alexandra Alexis,Duran Castillo,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True,None,a1W8c000008TCWxEAO
7936,165940962,Hombre,.,francisco.alcota.h@gmail.com,972087289,Francisco Andres,Alcota Hidalgo,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True,None,a1W8c000008TCWxEAO
7937,182046884,Mujer,,,933141806,Yamilet Margarita,Altamirano Maldonado,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,False,None,a1W8c000008TCWxEAO
7938,160031239,Hombre,Diego Portales 860,xmaquinax@gmail.com,975084179,Torres Valderrama,Patricio Joaquin,Chile,RUT,a1U8c000005dAtnEAE,0128c000002Byp4AAC,True,Validada,True,None,a1W8c000008TCWxEAO


In [22]:
prueba = "153298939"
sf.query(
    format_soql(
        "SELECT Id,lastname FROM Account WHERE IdentificationId__pc = {last_name}",
        last_name=prueba,
    )
)

OrderedDict([('totalSize', 1),
             ('done', True),
             ('records',
              [OrderedDict([('attributes',
                             OrderedDict([('type', 'Account'),
                                          ('url',
                                           '/services/data/v42.0/sobjects/Account/0018c00002anTmeAAE')])),
                            ('Id', '0018c00002anTmeAAE'),
                            ('LastName', 'Martinez Matus')])])])

In [23]:
""" sf.Account.create(
    {
        "IdentificationId__pc": "12345678910",
        "HealthCloudGA__Gender__pc": "Mujer",
        "FirstName": "johan",
        "LastName": "Smith",
        "PersonEmail": "example@example.com",
        "BillingCountry": "Chile",
        "Identification_Type__c": "RUT",
        "Pa_s_del_Paciente__c": "a1U8c000005dAtnEAE",
        "RecordTypeId": "0128c000002Byp4AAC",
    }
)
 """

# sf.Account.update(1, {'Name': 'Test Account', ContactId: contact_id}) # doctest: +SKIP

' sf.Account.create(\n    {\n        "IdentificationId__pc": "12345678910",\n        "HealthCloudGA__Gender__pc": "Mujer",\n        "FirstName": "johan",\n        "LastName": "Smith",\n        "PersonEmail": "example@example.com",\n        "BillingCountry": "Chile",\n        "Identification_Type__c": "RUT",\n        "Pa_s_del_Paciente__c": "a1U8c000005dAtnEAE",\n        "RecordTypeId": "0128c000002Byp4AAC",\n    }\n)\n '

In [34]:
""" import pandas as pd
sf.bulk.Account.query("SELECT Id, Email FROM Account")

df = pd.DataFrame.from_dict(data,orient='columns').drop('attributes',axis=1) """

import pandas as pd

df = pd.DataFrame([{'Id':1},{'Id':2},{'Id':3}])
def dataframe_to_sfdc_list(df,column):
    df_list = df[column].unique()
    df_list = [str(x) for x in df_list]
    df_list = ','.join("'"+item+"'" for item in df_list)
    return df_list

sf.query(format_soql("SELECT Id, Phone FROM Account WHERE Id IN ({})", dataframe_to_sfdc_list(df,column)))


NameError: name 'column' is not defined

In [ ]:
contact = sf.Account.get('0018c00002amLcCAAU')

In [25]:
# sf = simple_salesforce.Salesforce(...)

query = "SELECT Id, clinica__c,AccountId,CreatedDate,RUT__c,Account.IdentificationId__pc, Work_TypeGroup__c,Status,Interacci_n__c,AppointmentNumber,CreatedById,Tipo_de_la_cita__c,Empresa_Principal__c,Country from ServiceAppointment"

df = sf.smart_query(query)
print(df)

100%|██████████| 1/1 [00:18<00:00, 18.84s/it]

                       Id           AccountId    CreatedDate     RUT__c  \
0      08p8c0000007756AAA  0018c00002ay9PHAAY  1668868174000  150066417   
1      08p8c000000775GAAQ  0018c00002amLcAAAU  1668870893000   78143835   
2      08p8c000000775LAAQ  0018c00002amLcVAAU  1668871031000  18674344K   
3      08p8c0000007769AAA  0018c00002amLcZAAU  1668873317000  106879508   
4      08p8c00000077CcAAI  0018c00002bJgIaAAK  1669045284000  248270454   
...                   ...                 ...            ...        ...   
44745  08pRN000000BbQrYAK  0018c00002ao2v0AAA  1704288815000  139113454   
44746  08pRN000000BbSTYA0  0018c00002amQXNAA2  1704288916000  145970253   
44747  08pRN000000BbVhYAK  001RN000000kxRAYAY  1704289072000  20467910K   
44748  08pRN000000BbXJYA0  0018c00002giNGdAAM  1704289182000  197438274   
44749  08pRN000000BbaXYAS  001RN000003SoQsYAK  1704289609000  172552404   

      Account.IdentificationId__pc   Work_TypeGroup__c      Status  \
0                        1500

In [26]:
df.head(-50)

,Id,AccountId,CreatedDate,RUT__c,Account.IdentificationId__pc,Work_TypeGroup__c,Status,Interacci_n__c,AppointmentNumber,CreatedById,Tipo_de_la_cita__c,Empresa_Principal__c,Country
0,08p8c0000007756AAA,0018c00002ay9PHAAY,1668868174000,150066417,150066417,0VS8c0000005YaAGAU,Programado,Ingreso,SA-0030,0058c00000DcdNgAAJ,Phone,-,Chile
1,08p8c000000775GAAQ,0018c00002amLcAAAU,1668870893000,78143835,78143835,0VS8c0000005YaAGAU,Ninguno,Seguimiento,SA-0032,0058c00000DcdNgAAJ,Phone,-,Chile
2,08p8c000000775LAAQ,0018c00002amLcVAAU,1668871031000,18674344K,18674344K,0VS8c0000005YZvGAM,Ninguno,Ingreso,SA-0033,0058c00000DcdO3AAJ,Phone,-,Chile
3,08p8c0000007769AAA,0018c00002amLcZAAU,1668873317000,106879508,106879508,0VS8c0000005YaeGAE,Ninguno,Seguimiento,SA-0045,0058c00000DcdNkAAJ,Phone,0018c00002axQecAAE,Chile
4,08p8c00000077CcAAI,0018c00002bJgIaAAK,1669045284000,248270454,248270454,0VS8c0000005YaAGAU,Cancelado,Ingreso,SA-0059,0058c00000DcdNhAAJ,Phone,-,Chile
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44695,08pRN000000BXALYA4,0018c00002giOmZAAU,1704224362000,13848145K,13848145K,0VS8c000000ob1yGAA,Confirmado,Ingreso MetLife,SA-50923,0058c00000Bpv8HAAR,Video,0018c00002giMlpAAE,Chile
44696,08pRN000000BXFBYA4,0018c00002amOj3AAE,1704224606000,253032529,253032529,0VS8c000000obNXGAY,Programado,Seguimiento MetLife,SA-50924,0058c00000DcdO5AAJ,Video,0018c00002cXcjLAAS,Chile
44697,08pRN000000BXK1YAO,0018c00002cWZLfAAO,1704225539000,179970988,179970988,0VS8c000000obNXGAY,Programado,Seguimiento MetLife,SA-50925,0058c00000DcdO3AAJ,Phone,0018c00002cXJhrAAG,Chile
44698,08pRN000000BXYXYA4,001RN000003hClnYAE,1704226284000,IAPD030401HDFSRVA3,IAPD030401HDFSRVA3,0VS8c000000HKSCGA4,Programado,Ingreso,SA-50926,0058c00000DT4ckAAD,Phone,-,México


In [30]:
rut = "153298939"

""" busquedaDatos = (
    "FIND {" + rut + "} "
) """
derty = sf.quick_search("153298939")
derty


OrderedDict([('searchRecords',
              [OrderedDict([('attributes',
                             OrderedDict([('type', 'Account'),
                                          ('url',
                                           '/services/data/v42.0/sobjects/Account/0018c00002anTmeAAE')])),
                            ('Id', '0018c00002anTmeAAE')]),
               OrderedDict([('attributes',
                             OrderedDict([('type', 'Receta__c'),
                                          ('url',
                                           '/services/data/v42.0/sobjects/Receta__c/a4DRN000000M7Z72AK')])),
                            ('Id', 'a4DRN000000M7Z72AK')]),
               OrderedDict([('attributes',
                             OrderedDict([('type', 'Receta__c'),
                                          ('url',
                                           '/services/data/v42.0/sobjects/Receta__c/a4DRN000000M7cL2AS')])),
                            ('Id', 'a4DRN000000M7cL2

In [31]:
df = pd.DataFrame([{'Id':1},{'Id':2},{'Id':3}])
archivoBCI["IdentificationId__pc"]
df.head()

,Id
0,1
1,2
2,3


In [32]:
def dataframe_to_sfdc_list(df,column):
    df_list = df[column].unique()
    df_list = [str(x) for x in df_list]
    df_list = ','.join("'"+item+"'" for item in df_list)
    return df_list

sf.query(format_soql("SELECT Id FROM Account WHERE IdentificationId__pc IN ({})", dataframe_to_sfdc_list(pacienteProgr,"sfRut")))

SalesforceGeneralError: Error Code 431. Response content: Request Header Fields Too Large

In [ ]:
pacienteProgr.head()

Codigo que dio github Copilot para la solucion del error de 0 en envio de correos

In [ ]:
import win32com.client as win32
import os
import shutil

nombre_reporte = "Vida Security - Ferronor"
nombre_carpeta = "F:/Reportes SF/Vida Security/"
nombre_archivo_final = nombre_carpeta + str(dia) + "/" + nombre_reporte + ".xlsx"
Archivo_origen = nombre_carpeta + "Datos/" + nombre_reporte + ".xlsx"
crear_carpeta(nombre_carpeta)
actualizar(Archivo_origen)
Asunto = "Detalle de Uso " + nombre_reporte + " - " + str(dia)
shutil.copy2(Archivo_origen, nombre_archivo_final, follow_symlinks=True)

# create Excel application object
excel = win32.gencache.EnsureDispatch('Excel.Application')

# set ScreenUpdating property to False to hide Excel application
excel.ScreenUpdating = False

# open workbook
workbook = excel.Workbooks.Open(nombre_archivo_final)

# do some work with the workbook

# save and close workbook
workbook.Save()
workbook.Close()

# set ScreenUpdating property to True to show Excel application
excel.ScreenUpdating = True

# quit Excel application
excel.Quit()

# release COM object
del excel

envio_de_email(
    destinatarios=["marialaura.perez@security.cl",],
    concopia=[
        "dnunez@careassistance.com",
        "dgarcia@careassistance.com",
        "pcerda@careassistance.com",
        "frubio@careassistance.com",
    ],
    asunto=Asunto,
    nombre_reporte=str(nombre_reporte),
    ruta_adjunto=nombre_archivo_final,
    nombre_adjunto=nombre_reporte + ".xlsx",
    activo=1,
)

In [ ]:
import win32com.client as win32
import os
import shutil
import datetime as dt

def crear_carpeta(nombre_carpeta):
    if not os.path.exists(nombre_carpeta):
        os.makedirs(nombre_carpeta)

def actualizar(Archivo_origen):
    excel = win32.gencache.EnsureDispatch('Excel.Application')
    wb = excel.Workbooks.Open(Archivo_origen)
    wb.RefreshAll()
    wb.Save()
    wb.Close()

def envio_de_email(destinatarios, concopia, asunto, nombre_reporte, ruta_adjunto, nombre_adjunto):
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = ";".join(destinatarios)
    mail.CC = ";".join(concopia)
    mail.Subject = asunto
    mail.Body = "Buen día,\n\nSe adjunta el detalle de uso de la aplicación " + nombre_reporte + " correspondiente al día de ayer.\n\nSaludos cordiales,\n\n"
    mail.Attachments.Add(ruta_adjunto, DisplayName=nombre_adjunto)
    mail.Send()

nombre_reporte = "Zurich - AMSA"
nombre_carpeta = "F:/Reportes SF/Zurich/"
nombre_archivo_final = nombre_carpeta + str(dt.date.today()) + "/" + nombre_reporte + ".xlsx"
Archivo_origen = nombre_carpeta + "Datos/" + nombre_reporte + ".xlsx"
crear_carpeta(nombre_carpeta)
actualizar(Archivo_origen)
Asunto = "Detalle de Uso " + nombre_reporte + " - " + str(dt.date.today())
shutil.copy2(Archivo_origen, nombre_archivo_final, follow_symlinks=True)

envio_de_email(
    destinatarios=[
        "clara.angelica.munoz@zurich.com",
        "claudia.ivonne.zapata@zurich.com",
        "maria.moreno.poblete@zurich.com",
        "nadia.pino@zurich.com",
    ],
    concopia=[
        "dgarcia@careassistance.com",
        "dnunez@careassistance.com",
        "pcerda@careassistance.com",
        "frubio@careassistance.com",
    ],
    asunto=Asunto,
    nombre_reporte=str(nombre_reporte),
    ruta_adjunto=nombre_archivo_final,
    nombre_adjunto=nombre_reporte + ".xlsx"